<a href="https://colab.research.google.com/github/Mahe-faure/db_testing/blob/main/Week13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('example.db')
c = conn.cursor()

# Create a sample table
c.execute('''CREATE TABLE IF NOT EXISTS employees (id INTEGER PRIMARY KEY, name TEXT, department TEXT, salary INTEGER)''')

# Example 1: Efficient Indexing
# Description: This example demonstrates how to create an index on the 'department' column.
# The index will improve the performance of queries filtering by department

# Adding some sample data
sample_data = [(1, 'Alice', 'HR', 70000), (2, 'Bob', 'Engineering', 80000), (3, 'Charlie', 'HR', 75000)]
c.executemany('INSERT INTO employees VALUES(?,?,?,?)', sample_data)

# Creating an index
c.execute('CREATE INDEX idx_department ON employees (department)')

# Comment: Indexes can significantly improve the performance of queries that filter or sort based on the indexed column.
# However, they can also slow down data insertion and updates. Therefore, use them judiciously.

# Querying with and without the index
c.execute('SELECT * FROM employees WHERE department = "HR"')
print("Query plan with index:", c.fetchall())

# Drop the index
c.execute('DROP INDEX idx_department')

c.execute('SELECT * FROM employees WHERE department = "HR"')
print("Query plan without index:", c.fetchall())

# CLeanup
conn.commit()
conn.close()

Query plan with index: [(1, 'Alice', 'HR', 70000), (3, 'Charlie', 'HR', 75000)]
Query plan without index: [(1, 'Alice', 'HR', 70000), (3, 'Charlie', 'HR', 75000)]


In [18]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('example.db')
c = conn.cursor()

# Create a sample table
c.execute('''CREATE TABLE IF NOT EXISTS products (id INTEGER PRIMARY KEY, name TEXT, category TEXT, price INTEGER)''')

# Adding some sample data
sample_data = [(1, 'Laptop', 'Electronics', 1200), (2, 'Chair', 'Furniture', 150), (3, 'Smartphone', 'Electronics', 800)]
c.executemany('INSERT INTO products VALUES (?,?,?,?)', sample_data)

# Query optimization example
# ---------------------------
# Description: This example demonstrates how to write an optimized query by selecting only the necessary columns
# and using efficient filtering

# Efficient query: selecting specific columns and filtering
c.execute('SELECT name, price FROM products WHERE category = "Electronics"')
print("Efficient query results: ", c.fetchall())

# Cleanup
conn.commit()
conn.close()

Efficient query results:  [('Laptop', 1200), ('Smartphone', 800)]


In [19]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('example.db')
c = conn.cursor()

# Create a sample table
c.execute('''CREATE TABLE IF NOT EXISTS orders (id INTEGER PRIMARY KEY, product_id INTEGER, quantity INTEGER)''')

# Transaction optimization example
# -------------------------------
# Description: This example demonstrates the uses of transactions to optimize bulk data insertions.

# Start a transaction
conn.execute('BEGIN TRANSACTION')

# Bulk insert data
bulk_data = [(i, i % 5 + 1, i * 2) for i in range(1, 1001)]
c.executemany('INSERT INTO orders VALUES (?,?,?)', bulk_data)

# Commit the transaction
conn.commit()

# Query to verify daata insertion
c.execute('SELECT COUNT(*) FROM orders')
print("Number of records inserted:", c.fetchone()[0])

# Cleanup
conn.close()

Number of records inserted: 1000


In [20]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('example.db')
c = conn.cursor()

# Create a sample table
c.execute('''CREATE TABLE IF NOT EXISTS students (id INTEGER PRIMARY KEY, name TEXT, class_id INTEGER)''')
c.execute('''CREATE TABLE IF NOT EXISTS classes (id INTEGER PRIMARY KEY, class_name TEXT)''')

# Adding some sample data
students_data = [(1, 'John', 1), (2, 'Emma', 2), (3, 'Harry', 1)]
classes_data = [(1, 'Math'), (2, 'Science',)]
c.executemany('INSERT INTO students VALUES (?,?,?)', students_data)
c.executemany('INSERT INTO classes VALUES (?,?)', classes_data)

# Efficient Joins example
# ---------------------------
# Description: This example demonstrates how to perform an efficient join between two tables.

# Efficient join query
c.execute('''SELECT students.name, classes.class_name
              FROM students
              JOIN classes ON students.class_id = classes.id''')
print("Join results: ", c.fetchall())

# Cleanup
conn.commit()
conn.close()

Join results:  [('John', 'Math'), ('Emma', 'Science'), ('Harry', 'Math')]


In [21]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('example.db')
c = conn.cursor()

# Advanced PRAGMA Optimization Example
# ------------------------------------
# Description: This example demonstrates the advanced use of PRAGMA statements
# for various database optimizations including write-ahead logging, foreign key enforcement,
# and query performance improvement.

# 1, Setting the journal mode to WAL (Write-Ahead Logging)
# WAL can improve the performance of read operations while write operations are happening.
c.execute('PRAGMA journal_mode=WAL')
journal_mode= c.fetchone()[0]
print ("Journal mode set to:", journal_mode)

# 2. Enabling foreign key constraints
# Ensures referential integrity in the database,
c.execute('PRAGMA foreign_keys=ON')
c.execute('PRAGMA foreign_keys')
fk_status = c.fetchone()[0]
print ("Foreign keys status:", "Enabled" if fk_status else "Disabled")

# 3. Setting synchronous mode to NORMAL
# This reduces the number of disk writes, speeding up transactions at the risk of some data loss in case of a crash,
c.execute('PRAGMA synchronous=NORMAL')
c.execute('PRAGMA synchronous')
sync_mode = c.fetchone()[0]
print ("Synchronous mode set to:", sync_mode)

# 4. Adjusting cache size
# Increasing the cache size can improve the database read performance by reducing disk 1/0,
c.execute( 'PRAGMA cache_size=10000') # Setting cache size to 10000 pages
c.execute('PRAGMA cache_size')
cache_size = c.fetchone()[0]
print ("Cache size set to:", cache_size, "pages")

# Useful for monitoring the database size, which can be a factor in performance.
# 5, Querying the database file size.
c.execute( 'PRAGMA page_size')
page_size = c.fetchone()[0]
c.execute('PRAGMA page_count')
page_count = c.fetchone()[0]
db_size = page_size * page_count
print ("Database size:", db_size, "bytes")

# Cleanup
conn.close()

Journal mode set to: wal
Foreign keys status: Enabled
Synchronous mode set to: 1
Cache size set to: 10000 pages
Database size: 36864 bytes
